<a href="https://colab.research.google.com/github/juanmanuel782/poo-2025A/blob/main/aerofly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import random
from typing import List, Dict, Optional

class Pasajero:
    def __init__(self, nombre: str, pasaporte: str, contacto: str):
        self.nombre = nombre
        self.pasaporte = pasaporte
        self.contacto = contacto

    def mostrar_informacion(self) -> str:
        return f"Nombre: {self.nombre}\nPasaporte: {self.pasaporte}\nContacto: {self.contacto}"

class Vuelo:
    def __init__(self, destino: str, fecha_hora: datetime.datetime, numero_vuelo: str, asientos_totales: int):
        self.destino = destino
        self.fecha_hora = fecha_hora
        self.numero_vuelo = numero_vuelo
        self.asientos_totales = asientos_totales
        self.asientos_disponibles = asientos_totales
        self.pasajeros: List[Pasajero] = []

    def reservar_asiento(self, pasajero: Pasajero) -> bool:
        if self.asientos_disponibles > 0:
            self.pasajeros.append(pasajero)
            self.asientos_disponibles -= 1
            return True
        return False

    def cancelar_reserva(self, pasajero: Pasajero) -> bool:
        if pasajero in self.pasajeros:
            self.pasajeros.remove(pasajero)
            self.asientos_disponibles += 1
            return True
        return False

    def mostrar_informacion(self) -> str:
        return f"Vuelo {self.numero_vuelo}\nDestino: {self.destino}\nFecha y Hora: {self.fecha_hora.strftime('%Y-%m-%d %H:%M')}\nAsientos disponibles: {self.asientos_disponibles}/{self.asientos_totales}"

class Reserva:
    def __init__(self, pasajero: Pasajero, vuelo: Vuelo, asientos_reservados: int):
        self.pasajero = pasajero
        self.vuelo = vuelo
        self.asientos_reservados = asientos_reservados
        self.codigo_reserva = self._generar_codigo()

    def _generar_codigo(self) -> str:
        return f"RES-{random.randint(1000, 9999)}-{self.vuelo.numero_vuelo}"

    def mostrar_informacion(self) -> str:
        return f"Código de reserva: {self.codigo_reserva}\nPasajero: {self.pasajero.nombre}\nVuelo: {self.vuelo.numero_vuelo} a {self.vuelo.destino}\nFecha: {self.vuelo.fecha_hora.strftime('%Y-%m-%d')}\nAsientos: {self.asientos_reservados}"

    def cancelar(self) -> bool:
        return self.vuelo.cancelar_reserva(self.pasajero)

class EuroFly:
    DIAS_OPERATIVOS = ["Monday", "Wednesday", "Friday"]
    HORARIOS_SALIDA = [datetime.time(5, 0), datetime.time(20, 0)]

    def __init__(self):
        self.nombre = "EuroFly"
        self.destinos: List[str] = []
        self.vuelos: List[Vuelo] = []
        self.reservas: List[Reserva] = []

    def agregar_destino(self, destino: str) -> None:
        if destino not in self.destinos:
            self.destinos.append(destino)

    def eliminar_destino(self, destino: str) -> bool:
        if destino in self.destinos:
            vuelos_destino = [v for v in self.vuelos if v.destino == destino and v.fecha_hora > datetime.datetime.now()]
            if vuelos_destino:
                return False
            self.destinos.remove(destino)
            return True
        return False

    def mostrar_destinos(self) -> str:
        return "\n".join(f"{i+1}. {destino}" for i, destino in enumerate(self.destinos))

    def crear_vuelo(self, destino: str, fecha: datetime.date, horario: datetime.time, asientos: int = 150) -> bool:
        if destino not in self.destinos:
            return False
        if fecha.strftime("%A") not in self.DIAS_OPERATIVOS:
            return False
        if horario not in self.HORARIOS_SALIDA:
            return False
        fecha_hora = datetime.datetime.combine(fecha, horario)
        for vuelo in self.vuelos:
            if vuelo.destino == destino and vuelo.fecha_hora == fecha_hora:
                return False
        numero_vuelo = f"EF-{random.randint(100, 999)}"
        self.vuelos.append(Vuelo(destino, fecha_hora, numero_vuelo, asientos))
        return True

    def buscar_vuelos(self, destino: Optional[str] = None, fecha: Optional[datetime.date] = None) -> List[Vuelo]:
        ahora = datetime.datetime.now()
        return [v for v in self.vuelos if v.fecha_hora > ahora and (not destino or v.destino == destino) and (not fecha or v.fecha_hora.date() == fecha)]

    def cancelar_vuelo(self, numero_vuelo: str) -> bool:
        for vuelo in self.vuelos:
            if vuelo.numero_vuelo == numero_vuelo and vuelo.fecha_hora > datetime.datetime.now():
                self.reservas = [r for r in self.reservas if not (r.vuelo == vuelo and r.cancelar())]
                self.vuelos.remove(vuelo)
                return True
        return False

    def hacer_reserva(self, pasajero: Pasajero, vuelo: Vuelo, asientos: int = 1) -> Optional[Reserva]:
        if vuelo not in self.vuelos or vuelo.fecha_hora <= datetime.datetime.now():
            return None
        if asientos <= 0 or asientos > vuelo.asientos_disponibles:
            return None
        for _ in range(asientos):
            if not vuelo.reservar_asiento(pasajero):
                return None
        reserva = Reserva(pasajero, vuelo, asientos)
        self.reservas.append(reserva)
        return reserva

    def cancelar_reserva(self, codigo_reserva: str) -> bool:
        for reserva in self.reservas:
            if reserva.codigo_reserva == codigo_reserva:
                if reserva.cancelar():
                    self.reservas.remove(reserva)
                    return True
        return False
